## Constants in Pluto

repo: https://github.com/privacy-scaling-explorations/halo2curves/tree/main/src/pluto_eris , https://github.com/daira/pluto-eris \
paper: https://eprint.iacr.org/2010/354.pdf

Given an integer $n < 2^{448}$, return its 7 limbs where the first limb is the least significant one.\
Each limb is 64 bits long in big endian.

In [15]:
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (112 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 112

    res = []
    for i in range(7):
        temp = '0x' + full_width_str[112 - 16 * (i + 1) : 112 - 16 * i]
        res.append(temp)
    return res

Convert a given integer into its little endian NAF, to make the Hamming weight minimal, \
in other words, to get out the chunk of power of two as large as possible (the bit may be $-1$):
$$x = 2^{n_0} - 2^{n_1} + \dots$$
Definition of NAF: https://en.wikipedia.org/wiki/Non-adjacent_form

In [16]:

def naf(n):
    bits = []
    while n > 0:
        if n % 2 == 1:
            # If n is odd, set the current digit as 2 - (n mod 4).
            digit = 2 - (n % 4)
            n -= digit
        else:
            # If n is even, set the current digit as 0.
            digit = 0
        bits.append(digit)
        n //= 2
    return bits

# Example usage:
# decimal_number = 7
# naf = naf(decimal_number)
# print(naf): [-1, 0, 0, 1], the binary form is initially [1, 1, 1] = 1 + 2 + 4.

In [17]:
print(naf(7))

[-1, 0, 0, 1]


Given a NAF, output the corresponding minimal Hamming weight representation

In [18]:
def low_hamming_rep(naf):
    l = len(naf)
    value = 0
    hamming_bits = {}
    for i in range(l):
        if naf[i] != 0:
            hamming_bits[i] = naf[i]
            value = value + naf[i] * 2**i
    # sanity check
    new_value = 0
    for index, bit in hamming_bits.items():
        new_value = new_value + bit * 2**index
    assert new_value == value, "the decomposition is not correct"
    return hamming_bits

In [19]:
print(low_hamming_rep(naf(7)))

{0: -1, 3: 1}


### The Base Field $\mathbb{F}_p$
seed $U = -1298074214633708060054710657220608$ \
the base field size $p = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1$ \
we define the constant $NEG\_U = -U$:

In [20]:
U = -1298074214633708060054710657220608
NEG_U = -U
NEG_U.bit_length()

111

In [21]:
hex(NEG_U)

'0x4000000000001000008780000000'

The minimal Hamming weight representation of $-U$ followed by the next computation:
$$-U = 2^{110} + 2^{60} + 2^{39} + 2^{35} - 2^{31}$$

In [22]:
print(low_hamming_rep(naf(NEG_U)))

{31: -1, 35: 1, 39: 1, 60: 1, 110: 1}


As the base field size $p = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1$ \
we compute the minimal Hamming weight representation of $p$ using symbolic computation:

In [23]:
R_over_q = PolynomialRing(QQ, 't')  # 't' stands for two
t = R_over_q.gen() # Don't forget to explicit the generator of ring
poly_U = -(t^(110) + t^(60) + t^(39) + t^(35) - t^(31))
# p = 9 * 2^2 * U^4 + 9 * 2^2 * U^3 + 3 * 2^3 * U^2 + 3 * 2^1 * U + 1
poly_p = 9 * t^2 * poly_U^4 + 9 * t^2 * poly_U^3 + 3 * t^3 * poly_U^2 + 3 * t * poly_U + 1
poly_p

9*t^442 + 36*t^392 + 36*t^371 + 36*t^367 - 36*t^363 + 54*t^342 - 9*t^332 + 108*t^321 + 108*t^317 - 108*t^313 + 54*t^300 + 108*t^296 - 18*t^292 - 108*t^288 + 54*t^284 - 27*t^282 + 108*t^271 + 108*t^267 - 108*t^263 - 27*t^261 - 27*t^257 + 27*t^253 + 108*t^250 + 216*t^246 - 99*t^242 - 216*t^238 + 108*t^234 - 27*t^232 + 36*t^229 + 108*t^225 + 3*t^223 + 36*t^221 - 144*t^217 - 36*t^213 - 54*t^211 + 108*t^209 - 54*t^207 - 36*t^205 + 54*t^203 + 54*t^200 + 108*t^196 - 54*t^192 - 27*t^190 - 108*t^188 - 54*t^186 + 54*t^184 + 18*t^182 + 36*t^179 + 54*t^178 + 108*t^175 - 27*t^174 + 6*t^173 - 180*t^167 - 27*t^161 + 108*t^159 + 9*t^158 - 27*t^157 - 36*t^155 + 36*t^154 + 27*t^153 + 6*t^152 + 18*t^150 + 6*t^148 - 72*t^146 - 6*t^144 - 45*t^142 - 27*t^140 + 72*t^138 - 54*t^136 + 18*t^134 + 27*t^132 - 36*t^130 + 54*t^128 + 9*t^126 - 27*t^124 + 3*t^123 - 9*t^119 - 27*t^115 - 3*t^111 + 45*t^107 + 6*t^102 - 27*t^99 + 6*t^98 + 9*t^95 - 6*t^94 + 3*t^81 + 6*t^77 - 3*t^73 - 6*t^69 + 3*t^65 - 3*t^61 - 3*t^40 - 3*

$p = 9 \cdot 2^{442} + 9 \cdot 2^{394} + 9 \cdot 2^{373} + 9 \cdot 2^{369} + \dots $

The constant $- (6U + 2)$

In [24]:
NEG_SIX_U_PLUS_2 = -(6 * U + 2)

In [25]:
hex(NEG_SIX_U_PLUS_2)

'0x18000000000006000032cfffffffe'

$-(6U + 2)$ in little endian the NAF
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L22-L27

In [26]:
print(naf(NEG_SIX_U_PLUS_2))

[0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, -1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1]


The length of the list used to store NEG_SIX_U_PLUS_2 \
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L22


In [27]:
len(naf(NEG_SIX_U_PLUS_2))

114

In [28]:
print(low_hamming_rep(naf(NEG_SIX_U_PLUS_2)))

{1: -1, 32: 1, 34: -1, 36: -1, 38: 1, 40: -1, 42: 1, 61: -1, 63: 1, 111: -1, 113: 1}


Due to the existence of $-1$, the length of NAF is bigger than the usual bit length:

In [29]:
NEG_SIX_U_PLUS_2.bit_length()

113

the base field prime $p$ in terms of $U$:

In [30]:
p_int = 36 * U**4 + 36 * U**3 + 24 * U**2 + 6 * U + 1
hex(p_int)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001'

In [31]:
p_int

102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

In [32]:
c = 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913
c.bit_length()

446

In [33]:
2**448 - p_int

624627028691536808436752742380249437599065440477694538042099964325148608279121808075475032395645009332853082409791364842647708441247743

In [34]:
p_int == 2**448 - 624627028691536808436752742380249437599065440477694538042099964325148608279121808075475032395645009332853082409791364842647708441247743

True

In [35]:
low_hamming_rep(naf(p_int))

{0: 1,
 32: -1,
 34: 1,
 36: 1,
 38: -1,
 40: 1,
 42: -1,
 61: 1,
 63: -1,
 65: -1,
 67: 1,
 70: 1,
 72: 1,
 75: -1,
 78: -1,
 80: -1,
 83: 1,
 96: -1,
 98: 1,
 102: -1,
 104: 1,
 107: 1,
 109: -1,
 115: 1,
 117: 1,
 119: 1,
 121: 1,
 125: 1,
 127: 1,
 129: -1,
 131: -1,
 133: -1,
 145: -1,
 148: -1,
 150: -1,
 156: 1,
 159: 1,
 161: -1,
 169: -1,
 171: 1,
 173: 1,
 175: -1,
 179: 1,
 181: 1,
 183: 1,
 186: 1,
 189: -1,
 194: -1,
 199: 1,
 202: -1,
 204: -1,
 207: 1,
 209: 1,
 211: 1,
 215: 1,
 217: 1,
 219: -1,
 221: -1,
 224: 1,
 226: -1,
 229: -1,
 231: 1,
 233: 1,
 235: -1,
 237: 1,
 239: -1,
 246: 1,
 250: 1,
 254: 1,
 256: -1,
 260: -1,
 263: 1,
 265: -1,
 267: 1,
 269: 1,
 272: 1,
 275: -1,
 278: 1,
 282: 1,
 284: -1,
 288: -1,
 291: -1,
 295: 1,
 297: 1,
 299: 1,
 302: -1,
 306: 1,
 315: 1,
 317: 1,
 319: 1,
 322: 1,
 325: -1,
 328: 1,
 332: -1,
 335: -1,
 343: -1,
 345: -1,
 348: 1,
 365: -1,
 368: 1,
 372: -1,
 374: 1,
 376: 1,
 394: 1,
 397: 1,
 442: 1,
 445: 1}

Check the group order $p-1$ is divisible by $6$ and $4$

In [36]:
p_int.bit_length()

446

In [37]:
type(p_int)

<class 'sage.rings.integer.Integer'>

In [38]:
(p_int - 1) % 6 == 0

True

In [39]:
(p_int - 1) % 4 == 0

True

The limbs representation of $p$:\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp.rs#L39-L47

In [40]:
u_64_little_endian(p_int)

['0x9ffffcd300000001',
 '0xa2a7e8c30006b945',
 '0xe4a7a5fe8fadffd6',
 '0x443f9a5cda8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

### The base field $\mathbb{F}_p$

In [41]:
Fp = GF(p_int)
Fp

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913

### The constant $R$ for Montgomery form
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp.rs#L67-L75

In [42]:
u_64_little_endian((2**448) % p_int)

['0xa000163afffffff9',
 '0x8d68a2aaffd0ef18',
 '0xbf6a760a123e0121',
 '0x2242c7760637089c',
 '0x67e576bf526ff2f5',
 '0xf7a9dfffa183e9bf',
 '0x03ffffffffff03ff']

In [43]:
R = Fp(2**448)
R

11356855067116315761326349333718857071609919219953404605758555192204529273465116571178922486933671965396447230942486297326349317046265

### The Scalar Field $\mathbb{F}_q$
the scalar field size $q = 36 U^4 + 36 U^3 + 18 U^2 + 6 U+1$

In [44]:
q_int = 36 * U**4 + 36 * U**3 + 18 * U**2 + 6 * U + 1
hex(q_int)

'0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001'

The limbs representation of $q$:\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fq.rs#L39-L47

In [45]:
u_64_little_endian(q_int)

['0x1ffffcd300000001',
 '0x9ca7e85d60050af4',
 '0xe4a775fe8e177fd6',
 '0x443f9a5c7a8a6c7b',
 '0xa803ca76f439266f',
 '0x0130e0000d7f70e4',
 '0x2400000000002400']

In [46]:
Fq = GF(q_int)
Fq

Finite Field of size 102211695604070082112571065507755096754575920209623522239390234855480569854275933742834077002685857629445612735086326265689167708028929

### The field $\mathbb{F}_{p^2}$ 
defined by $\mathbb{F}_{p^2} = \mathbb{F}_p[u] = \mathbb{F}_p[X] / (X^2 + 5)$ where $X^2 + 5$ is irreducible in $\mathbb{F}_p[X]$\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp2.rs#L14-L27 (TODO: comment)

Check $X^2 + 5$ is irreducible:

In [47]:
Rp = PolynomialRing(Fp, "x")
x = Rp.gen()
(x^2 + 5).is_irreducible()

True

Construct $\mathbb{F}_{p^2}$ from $\mathbb{F}_{p}$ in $u$ such that $u^2 + 5 = 0$

In [48]:
Fp2.<u> = Fp.extension(x^2 + 5)
Fp2

Finite Field in u of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2

Check the identity $u^2 + 5 = 0$ in $\mathbb{F}_{p^{2}}$

In [49]:
u**2 + 5 == 0

True

### The twisted curve (Triton) defined by: $E': Y^2 = X^3 + (u + 3)$
see the defintion in: https://github.com/daira/pluto-eris \
and justified in the repo: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/curve.rs#L100-L106


### The field $\mathbb{F}_{p^6}$ 
defined by $\mathbb{F}_{p^6} = \mathbb{F}_{p^2}[v] = \mathbb{F}_{p^2}[X] / (X^3 - 57/(u + 3))$ where $X^3 - 57/(u+3)$ is irreducible in $\mathbb{F}_{p^2}[X]$\

Note that the non-cubic residue $57/(u + 3)$ is not arbitrarily chosen. It is closely related to the equation of the curve:
$$E / \mathbb{F}_p: Y^2 = X^3 + 57$$
and the twisted curve :
$$E' / \mathbb{F}_{p^2}: Y^2 = X^3 + (u + 3)$$
More precisely, let $\xi = 57/(u + 3) \in \mathbb{F}_{p^2}$, the isomorphism is defined by
$$\Phi: E' \rightarrow E: \quad (x, y) \mapsto (x \cdot \xi^{1/3}, y \cdot \xi^{1/2})$$
and
$$\Phi^{-1}: E \rightarrow E': \quad (x, y) \mapsto (x / \xi^{1/3}, y / \xi^{1/2})$$
See more details in Section 3 regarding D-type twist in paper: http://indigo.ie/~mscott/twists.pdf

Suppose $\xi = 57/(u + 3) = c_1 \cdot u + c_0$ where $c_0, c_1 \in \mathbb{F}_p$

In [50]:
xi = 57 * (u + 3)**(-1)
xi

21902506200872160452693799751661806447409125759205040479869336040462288535916310410867718211025404905980594113530317090904334254435763*u + 36504177001453600754489666252769677412348542932008400799782226734103814226527184018112863685042341509967656855883861818173890424059624

$\xi^{(p^2 - 1)/2 }$ is order $2$

In [51]:
xi**((p_int ** 2 - 1)/2) == Fp2(-1)

True

the coordinate $c_0$ of $\xi$ in the basis $\{ 1, u\}$ in $\mathbb{F}_{p^2}$\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L15-L23

In [52]:
u_64_little_endian(36504177001453600754489666252769677412348542932008400799782226734103814226527184018112863685042341509967656855883861818173890424059624)

['0xddb6da4b5b6db6e8',
 '0x833bf7b35b701d98',
 '0x3f6072240ebe2483',
 '0x73cd928ee056022c',
 '0xce4a7f2a7bcb4495',
 '0xdbda9924971b3a9a',
 '0x0cdb6db6db6dc3b6']

the coordinate $c_1$ of $\xi$ in the basis $\{ 1, u\}$ in $\mathbb{F}_{p^2}$\
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L25-L33

In [53]:
u_64_little_endian(21902506200872160452693799751661806447409125759205040479869336040462288535916310410867718211025404905980594113530317090904334254435763)

['0xeb6db62d36db6db3',
 '0xb523fb0536dcde8e',
 '0x8c6d1148d5a5491b',
 '0x457b57ef5366ce1a',
 '0x489319197d79f5f3',
 '0xb71cc2492776bcc3',
 '0x07b6db6db6db756d']

the irreducible polynomial in $\mathbb{F}_{p^2}[X]$: $X^3 - \xi$

In [54]:
Rp2 = PolynomialRing(Fp2, "x")
x = Rp2.gen()
(x^3 - xi).is_irreducible()

True

construct the field $\mathbb{F}_{p^6} = \mathbb{F}_{p^2}[X]/(X^3 - \xi) = \mathbb{F}_{p^2}[v]$

In [55]:
Fp6.<v> = Fp2.extension(x^3 - xi)
Fp6

Univariate Quotient Polynomial Ring in v over Finite Field in u of size 102211695604070082112571065507755096754575920209623522239390234855490679834276115250716018318118556227909439196474813090886893187366913^2 with modulus v^3 + 80309189403197921659877265756093290307166794450418481759520898815028391298359804839848300107093151321928845082944495999982558932931150*u + 65707518602616481358081399254985419342227377277615121439608008121386865607748931232603154633076214717941782340590951272713002763307289

In [56]:
Fp6.is_field()

True

check the identity $v^3 - \xi = 0$

In [57]:
v**3 - xi == 0

True

### The field $\mathbb{F}_p^{12}$
$\mathbb{F}_{p^{12}} = \mathbb{F}_{p^6}[w] = \mathbb{F}_{p^6}[X] / (X^2 - v)$ where $X^2 - v$ is irreducible in $\mathbb{F}_{p^6}[X]$ \
We may use the following constructor to construct $\mathbb{F}_{p^{12}}$ as usual, but it turns out too slow:


In [58]:
# Rp6 = PolynomialRing(Fp6, "x")
# x = Rp6.gen()
# (x^2 - v).is_irreducible()
# Fp12 = Rp6.quotient(x^2 - v)
# Fp12

Instead, let's check it manually. It suffices to prove $X^2 - v$ is irreducible over $\mathbb{F}_{p^6}[X]$, in other words
$v$ is not a quadratic residue over $\mathbb{F}_{p^6}$. Otherwise, there is an element $p(v) = a_0 + a_1 \cdot v + a_2 \cdot v^2 \in \mathbb{F}_{p^6}$ such that
$$p(v)^2 = v \text{ over } \mathbb{F}_{p^6}$$
Equivalently, there exists $p(X) = a_0 + a_1 \cdot X + a_2 \cdot X^2 \in \mathbb{F}_{p^2}[X]$ such that
$$ X^3 - 57/(u + 3) \;| \; p(X)^2 - X \text{ over } \mathbb{F}_{p^2}[X]$$
It is clear that there is no such a $p(X) \in \mathbb{F}_{p^2}[X]$.


### The const FROBENIUS_COEFF_FP6_C1[1] 
used in the pairing: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L462

In fact, the list FROBENIUS_COEFF_FQ6_C1 = $[\gamma_{10}. \gamma_{11}, \dots, \gamma_{15}]$ such that for the Frobenius morphism $\pi$:
$$\pi^i(v) = \gamma_{1i} \cdot v \quad \text{ for } i = 0, 1, \dots, 5$$
where $\pi$ has order $6$ in the group $Gal(\mathbb{F}_{q^6} / \mathbb{F}_q)$ and $\mathbb{F}_{q^6} = \mathbb{F}_{q^{2}}[v]$.

Notice $\{ 1, v, v^2\}$ is a basis for the $\mathbb{F}_{q^2}$-vector space $\mathbb{F}_{q^6}$, regarding the images of $v^2$, the list FROBENIUS_COEFF_FQ6_C2 = $[\gamma_{20}. \gamma_{21}, \dots, \gamma_{25}]$ is defined by:
$$\pi^i(v^2) = \gamma_{2i} \cdot v^2 \quad \text{ for } i = 0, 1, \dots, 5$$

For $v = (57/(u + 3))^{1/3} = \xi^{1/3}$, we have
$$\pi(v) = \xi^{p/3} = \xi^{(p-1)/3} \cdot \xi^{1/3} = \xi^{(p-1)/3} \cdot v$$
hence $\gamma_{11} = \xi^{(p-1)/3}$.

This constant is defined: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L459

In [59]:
xi**((p_int-1)/3)

92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858*u + 51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611

Let $\gamma_{11} = \xi^{(p-1)/3} = c_1 \cdot u + c_0 \in \mathbb{F}_{p^2}$\
the coordinate $c_0$: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L461-L470

In [60]:
hex(51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611)

'0x120de97f024c55bc3bc0d351f4c70da1e3886170077a50986f93678bc921dcd5041bc4bb14cc42dc52e787634eccc335a001825382850d03'

TODO: the representation is different from BN256 which is in the field representation (with Montgomery const $R$)

In [61]:
u_64_little_endian(51260142370505185497351973260211965617805492386872277241342529142140236670911574395535108090223527812381417682871589681672697256676611)

['0xa001825382850d03',
 '0x52e787634eccc335',
 '0x041bc4bb14cc42dc',
 '0x6f93678bc921dcd5',
 '0xe3886170077a5098',
 '0x3bc0d351f4c70da1',
 '0x120de97f024c55bc']

the coordinate $c_1$: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L471-L480

In [62]:
hex(92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858)

'0x2096f3f804d973afd82becc2ef081b76132461908eadbe3da1a7f5502b7091965efa1ddf4658080413be1b7cd3c9ea0e2772fea378a9b322'

In [63]:
u_64_little_endian(92529011805995300781026747858635174615077510851648588563707597606861718583198168205651139531970571313972517572887035428707503415341858)

['0x2772fea378a9b322',
 '0x13be1b7cd3c9ea0e',
 '0x5efa1ddf46580804',
 '0xa1a7f5502b709196',
 '0x132461908eadbe3d',
 '0xd82becc2ef081b76',
 '0x2096f3f804d973af']

### The const FROBENIUS_COEFF_FP6_C1[2] 
used in the pairing: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L470

By the above discussion, we need to compute the constant $\gamma_{12}$ such that $\pi^2(v) = \gamma_{12} \cdot v$. It is clear
$$\pi \circ \pi(v) = \pi(\gamma_{11} \cdot v) = \overline{\gamma}_{11} \cdot \pi(v) = \overline{\gamma}_{11} \cdot \gamma_{11} \cdot v$$
since $\gamma_{11} \in \mathbb{F}_{p^2}$ and the Frobenius over $\mathbb{F}_{p^2}$ is equivalent to conjugation. Hence
$\gamma_{12} = \overline{\gamma}_{11} \cdot \gamma_{11} = \xi^{((p^2 - p) + p - 1)/3} = \xi^{(p^2 - 1)/3}$.

This constant can be found: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L482

In [64]:
xi**((p_int**2-1)/3)

39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334

$c_1 = 0$ and we compute $c_0$:\ 
https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/fields/fp6.rs#L484-L494

In [65]:
hex(39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334)

'0x480000000000360001c950000d7ee0e4a803c956d01c903d720dc8ad8b38dffaf50c100004c37ffffffe'

In [66]:
u_64_little_endian(39370513046094319542878173447389497729725081225711316008956793976697167703016365005507455943322894334)

['0x100004c37ffffffe',
 '0xc8ad8b38dffaf50c',
 '0xc956d01c903d720d',
 '0x50000d7ee0e4a803',
 '0x00000000360001c9',
 '0x0000000000004800',
 '0x0000000000000000']

### The const XI_TO_P_MINUS_1_OVER_2 
used in the pairing: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L29-L50


In [67]:
xi**((p_int-1)/2)

65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996*u + 95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166

the coordinate $c_0$

In [68]:
u_64_little_endian(95958235239618370663357236287665721330778695800924622118898462510960785934385719569712082879717746516106642479264739976661558948422166)

['0x54cf5ad1c0926216',
 '0x186c1f3ce4a46d4e',
 '0x9c23800ce9c9452f',
 '0x50e0d09ff6d6c08b',
 '0x7cf421e4d46f6666',
 '0x678664ba4b6d8343',
 '0x21cc26d5de0f80f4']

the coordinate $c_1$

In [69]:
u_64_little_endian(65079093581113076137070936836706683307023886770993453415929204320012344240499906559031972253911619563827341548210931723067336455786996)

['0xc0505f4c260e91f4',
 '0xe7bbd15f10723657',
 '0xb4b3e0c35358097e',
 '0x87c56f42a558750d',
 '0x4b7211d23f34f0ae',
 '0xf6839d29e2f0d250',
 '0x16ebe8b2e12a1106']

The order of $\xi$ is exactly $p^2 - 1$, thus $\xi^{(p^2 - 1)/2} = 1$. This fact is used in $y$ coordinate of 
the point $ -Q_2 = - \pi^2(Q)$.

In [70]:
xi**((p_int**2 - 1)/2) == -1

True

### Rationale of These Constants
These constants are used for computing $Q_1 = \pi(Q)$ and $Q_2 = \pi^2(Q)$.\
Recall in the implementation, $Q \in G_2$  as an affine point in twisted curve $E'(\mathbb{F}_{p^2})$. Suppose $Q = (x, y) \in E'(\mathbb{F}_{p^2})$, then
$$\Phi^{-1}: E' \longrightarrow E: (x, y) \mapsto (x \cdot \xi^{1/3}, y \cdot \xi^{1/2})$$
gives preimage of $Q$ in $E(\mathbb{F}_{p^{12}})$
$$(x \cdot \xi^{1/3}, y \cdot \xi^{1/2})$$
We then apply Frobenius in $E(\mathbb{F}_{p^{12}})$ as usual and map back to the twisted curve $E'(\mathbb{F}_{p^2})$:
$$E(\mathbb{F}_{p^{12}}) \xrightarrow{\pi} E(\mathbb{F}_{p^{12}}) \xrightarrow{\Phi^{-1}} E'(\mathbb{F}_{p^{2}})$$
$$(x \cdot \xi^{1/3}, y \cdot \xi^{1/2}) \mapsto (\overline{x} \cdot \xi^{p/3}, \overline{y} \cdot \xi^{p/2}) \mapsto (\overline{x} \cdot \xi^{(p-1)/3}, \overline{y} \cdot \xi^{(p-1)/2})$$
So $Q_1 = (\overline{x} \cdot \xi^{(p-1)/3}, \overline{y} \cdot \xi^{(p-1)/2})$, justified by: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/pluto_eris/engine.rs#L461-L467

### Test the final exponentiation

In [71]:
def f_exp_by_x(f, x):
    y = -x
    res = 1
    l = y.bit_length()
    for i in range(l-1, -1, -1):
        res = res**2
        if ((y>>i) & 1) == 1:
            res = res * f
    return 1/res

In [72]:
f_exp_by_x(2, -4)

1/16

### Test Miller loop

In [73]:
def test_miller_loop(L):
    l = len(L)
    res = 1
    vec = []
    for i in range(l-2, -1, -1):
        res = res*2
        vec.append(res)
        if L[i] == 1:
            res = res + 1
            vec.append(res)
        elif L[i] == -1:
            res = res - 1
            vec.append(res)
    return res
   
        
        

In [74]:
L = [-1, 0, 0, 1]
test_miller_loop(L)

7